**multi-link pendulum - python simulation and animation of a multi-link pendulum**

This jupyter notebook uses Lagrange formalism to derive the dynamics of a multi-link pendulum, simulates it and shows an animation of resulting solution. Unfortunately, it seems to be not very time-performant, so the best I could do in a reasonable amount of time was 6 links.

> [1] 1994 - Murray, Li, Sastry: *A Mathematical Introduction to Robotic Manipulation* (https://www.ce.cit.tum.de/fileadmin/w00cgn/rm/pdf/murray-li-sastry-94-complete.pdf)


Notes:
*   indexing starts with zero
*   angular positions and velocities are defined absolutely (in general joint based measurements of a link orientation is relative to the previous link)


(Benjamin Jahn, TU Ilmenau, 2021)

# Initilization

**initialization:** clear workspace, define latex use for display

In [22]:
# clear workspace
%reset -f

In [23]:
# latex math display
from IPython.display import Math, Latex

def latexDisplay(latex_obj): display(Math(latex_obj))
# def latexDisplay(latex_obj): display(Latex(r'$$' + (latex_obj) + r'$$'))

# Kinematics and Dynamics

**symbolic computation of the dynamics of a multi-link pendulum:** derive dynamics using Lagrange equations of the second kind [1, p.168ff]

In [24]:
# import sympy for symbolic computations
from sympy import *

In [25]:
# NUMBER OF LINKS (PUT THE DESIRED NUMBER OF LINKS HERE)
N = 25

In [28]:
# kinematic parameter
l = MatrixSymbol('l', N, 1); a = MatrixSymbol('a', N, 1)

# dynamic parameter
J = MatrixSymbol('J', N, 1); m = MatrixSymbol('m', N, 1); d = MatrixSymbol('d', N, 1)
grav = symbols('g', real=True, positive=True) # gravitational constant

# Plots

In [19]:
# plot imports
import plotly.graph_objs as go
import plotly.io as pio
pio.templates.default = "none"
pio.renderers.default = "notebook_connected"

In [20]:
# define generic plot function for angular position of triple pendulum
from colour import Color

def plot_pendulum(sol):
    start_color = '#0000ff'; end_color = '#00ff80'
    colorscale = [x.hex for x in list(Color(start_color).range_to(Color(end_color), N))]

    fig = go.Figure()
    for idx in range(0, N):
        fig.add_trace(go.Scatter(x=sol.t, y=sol.y[idx, :] / np.pi, name=r"$q_" + str(idx) + "$", 
            line=dict(width=2, color=colorscale[idx])))

    fig.update_layout(height=600, width=900, margin=go.layout.Margin(t=20), 
        xaxis=dict(title=r"$\text{time in seconds}$", mirror=True, ticks='outside', showline=True),
        yaxis=dict(title=r"$\text{angular position in } \pi \text{ rad}$", 
            tickmode="linear", tick0=0.0, dtick=1, mirror=True, ticks='outside', showline=True))

    fig.show()

In [21]:
# plot angular positions
plot_pendulum(sol)

# Animation

In [22]:
## import matplotlib for animations
import matplotlib.pyplot as plt
from matplotlib import animation, rc, cm
rc('animation', html='jshtml')

In [23]:
# define pendulum animation function
def animate_pendulum(sol, fps=25, colormap='winter'):

    colors = cm.get_cmap(colormap, N)

    # frame number and index distance for given fps (tried to avoid interpolation to not slow down animation even further)
    T2anim = sol.t[-1]
    frameNumber = fps*T2anim
    didx = floor(len(sol.t)/frameNumber)

    # create and setup figure
    fig, ax = plt.subplots(figsize=(14,10))
    plt.axis('equal'); plt.close()
    l_sum = sum(l_num)
    ax.set_xlim((-l_sum*1.65, l_sum*1.65)); ax.set_ylim((-l_sum*1.35, l_sum*1.35))

    # create line objects
    line_base, = ax.plot([], [], lw=3, c=(0, 0, 0))
    lines = [None]*N
    for idx in range(0,N):
        lines[idx], = ax.plot([], [], lw=4, c=colors(idx/N))

    # initialization function: plot the background of each frame
    def init():
        # draw rail
        line_base.set_data(np.linspace(-l_sum*1.65, l_sum*1.65, 10), 0*np.linspace(-2, 2, 10)) 
        
        return (line_base,*lines, ) # return line objects

    # animation function: this is called sequentially
    def animate(i):

        # current configuration of pendulum
        p = np.zeros((2,N+1))
        for idx in range(1,N+1):
            p[:,idx] = p[:,idx-1] + np.array([-l_num[idx-1]*sin(sol.y[idx-1,i*didx]), l_num[idx-1]*cos(sol.y[idx-1,i*didx])])

        for idx in range(0,N):
            lines[idx].set_data([p[0,idx], p[0, idx+1]],[p[1,idx], p[1, idx+1]])

        return (*lines,) # return line objects

    # return animation object
    return animation.FuncAnimation(fig, animate, init_func=init, frames=floor(frameNumber), interval=1/fps*1000, blit=True)


In [24]:
anim = animate_pendulum(sol) # no control input --> cart fixed in position
anim 

/tmp/ipykernel_31989/2608828685.py:4: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.



KeyboardInterrupt: 